<a id=top></a>

# Analysis of Conceptual Cell Archetypes

## Table of Contents

----

1. [Preparations](#prep)
2. [Mappings](#bmap)
3. [Grouped](#gplots)
4. [Prediction Probilities](#proba)
5. [Archetype Space](#archespace)

## Notes

----

- Selection criteria for manual cell annotation
    - `central rosette cells:` cells directly adjacent to a lumen without contact to the tissue's outside
    - `peripheral rosette cells:` cells to the left or right of a lumen with a large area of contact to the tissue's outside
    - `inter-organ cells:` cells between two rosettes, both within and on the outside of the tissue
    - `leader cells:` the first few cells from the front, in particular those without a clear apical backwards-polarity


- Numeric encoding of archetypes:
    - `0 : 'unclassified'`
    - `1 : 'central'`
    - `2 : 'outer'`
    - `3 : 'inter'`
    - `4 : 'leader'`

<a id=prep></a>

## 1. Preparations

----

In [ ]:
### Import modules

# External, general
from __future__ import division
import os, sys, pickle, warnings
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# External, specific
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
import scipy.stats as stats
from ipywidgets import interact
import ipywidgets as widgets

# Internal
import katachi.utilities.loading as ld

In [ ]:
### Load data

# Prep loader
loader = ld.DataLoaderIDR()
loader.find_imports(r"data/experimentA/extracted_measurements/", recurse=True, verbose=True)

# Import shape spaces
fspace_TFOR_pca, prim_IDs, fspace_idx = loader.load_dataset("shape_TFOR_pca_measured.tsv")
fspace_CFOR_pca, _, _ = loader.load_dataset("shape_CFOR_pca_measured.tsv", IDs=prim_IDs)
print "Imported TFOR shape space of shape:", fspace_TFOR_pca.shape
print "Imported CFOR shape space of shape:", fspace_CFOR_pca.shape

# Import TFOR centroid locations
centroids = loader.load_dataset("_other_measurements.tsv", IDs=prim_IDs)[0][:,3:6][:,::-1]
print "Imported TFOR centroids of shape:", centroids.shape

# Import & standardize engineered features
covar_df, _, _ = loader.load_dataset("_other_measurements.tsv", IDs=prim_IDs, force_df=True)
del covar_df['Centroids RAW X']; del covar_df['Centroids RAW Y']; del covar_df['Centroids RAW Z']
covar_names = list(covar_df.columns)
covar_df_z = (covar_df - covar_df.mean()) / covar_df.std()
print "Imported engineered features of shape:", covar_df.shape

In [ ]:
### Load archetype data

# Source fspace
archetype_type = 'TFOR'
#archetype_type = 'CFOR'

# Load archetype predictions
archetype_classes, _, _ = loader.load_dataset("_archetype_"+archetype_type+"_classifications.tsv", 
                                              IDs=prim_IDs)
print "Imported archetype classifications of shape:", archetype_classes.shape
archetype_probas, _, _  = loader.load_dataset("_archetype_"+archetype_type+"_probabilities.tsv", 
                                              IDs=prim_IDs)
print "Imported archetype probabilities of shape:", archetype_probas.shape

# Archetype annotation and visualization
archetype_decodedict = {0 : 'unclassified', 1 : 'central', 2 : 'peri',
                        3 : 'inter', 4 : 'leader'}
archetype_encodedict = {name:key for key,name in archetype_decodedict.iteritems()}
archetype_colors = {0 : 'lightgray', 1 : 'royalblue', 2 : 'limegreen',
                    3 : 'cyan', 4 : 'orangered'}

<a id=bmap></a>

## 2. Mappings

----

In [ ]:
### Tissue consensus map on centroids

# Axis range (easiest to set manually...)
xlimit = (-175, 15)
ylimit = (- 20, 20)
   
# Init
fig, ax = plt.subplots(1, figsize=(12,5))

# Back-mapping plot
for key in archetype_decodedict.keys():
    mask = archetype_classes==key
    if np.any(mask):
        scat = ax.scatter(centroids[mask, 2], centroids[mask, 1],
                          color=archetype_colors[key], edgecolor='', 
                          s=15, alpha=0.75, label=archetype_decodedict[key])

# Cosmetics
ax.set_xlim(xlimit)
ax.set_ylim(ylimit)
ax.invert_yaxis()  # To match images
ax.set_xlabel('TFOR x')
ax.set_ylabel('TFOR y')
ax.set_title('Centroid backmapping of predicted archetypes')
ax.legend(loc=2, frameon=False, fontsize=9)

# Done
plt.show()

In [ ]:
### Mapping onto shape space

# Set interactions
@interact(use_space=['TFOR','CFOR'],
          PCx=(1, fspace_TFOR_pca.shape[1], 1),
          PCy=(1, fspace_TFOR_pca.shape[1], 1))

# Show 
def show_PCA_backmap(use_space='TFOR', PCx=1, PCy=2): 
    
    # Select data
    if use_space=='TFOR':
        plot_fspace = fspace_TFOR_pca
    elif use_space=='CFOR':
        plot_fspace = fspace_CFOR_pca
    
    # Prep plot
    plt.figure(figsize=(9,7))
        
    # Create scatterplot
    for key in archetype_decodedict.keys():
        mask = archetype_classes==key
        if np.any(mask):
            scat = plt.scatter(plot_fspace[mask, PCx-1], plot_fspace[mask, PCy-1],
                               color=archetype_colors[key], edgecolor='', 
                               s=10, alpha=0.75, label=archetype_decodedict[key])

    # Cosmetics  
    plt.legend(frameon=False, fontsize=8)
    plt.xlabel(use_space+" PC "+str(PCx))
    plt.ylabel(use_space+" PC "+str(PCy))
    plt.title("Archetypes in "+use_space+" shape space")
    plt.show()

<a id=gplots></a>

## 3. Grouped Plots

----

In [ ]:
### Grouped shape space boxplot

# Set interactions
@interact(use_space=['TFOR','CFOR'],
          dim=(1, fspace_TFOR_pca.shape[1], 1))

# Create boxplot
def boxplot_fspaces(use_space='TFOR', dim=2):
    
    # Select data
    if use_space=='TFOR':
        plot_fspace = fspace_TFOR_pca
    elif use_space=='CFOR':
        plot_fspace = fspace_CFOR_pca
    
    # Group data
    ct_keys   = [key for key in archetype_decodedict.keys() if key!=0]
    plot_data = [plot_fspace[archetype_classes==key, dim-1] for key in ct_keys]
    
    # Prep plot
    fig, ax = plt.subplots(1, figsize=(6, 4))
    
    # Create boxplot
    bp = ax.boxplot(plot_data, widths=0.6, sym='', patch_artist=True)
    
    # Style boxplot
    for patch, color in zip(bp['boxes'], [archetype_colors[key] for key in ct_keys]):
        patch.set(color=color, alpha=0.5)
    for whisker in bp['whiskers']:
        whisker.set(color='black', linewidth=1.2, linestyle='-', alpha=0.5)
    for cap in bp['caps']:
        cap.set(linewidth=1.2, alpha=0.6)
    for median in bp['medians']:
        median.set(color='black', linewidth=1.2, alpha=0.5)
    
    # Add jittered data
    for i,p in enumerate(plot_data):
        y = p
        x = np.random.normal(i+1, 0.06, size=len(y))  # Jitter
        ax.plot(x, y, 'k.', alpha=0.1, ms=2)
    
    # Axis cosmetics
    ax.set_xticklabels([archetype_decodedict[key] for key in ct_keys],
                       rotation=45, ha='right')
    ax.set_ylabel(use_space+" PC "+str(dim))
    
    # Other cosmetics
    ax.set_title("Boxplot of fspace by archetype")
    
    # Done
    plt.show()

In [ ]:
### Grouped covariate boxplot

# Set interactions
@widgets.interact(covariate=covar_names,
                  standardized=['no','z'],
                  show_stats=False)

# Plot
def boxplot_covars(covariate='Sphericity',
                   standardized='no',
                   show_stats=False): 
    
    # Select covariate data
    if standardized=='no':
        covar_data = covar_df[covariate]
    elif standardized=='z':
        covar_data = covar_df_z[covariate]

    # Group data
    ct_keys   = [key for key in archetype_decodedict.keys() if key!=0]
    plot_data = [covar_data[archetype_classes==key] for key in ct_keys]
    
    # Prep plot
    fig, ax = plt.subplots(1, figsize=(5, 4))
    
    # Create boxplot
    bp = ax.boxplot(plot_data, widths=0.7, sym='', patch_artist=True)
    
    # Style boxplot
    for patch, color in zip(bp['boxes'], [archetype_colors[key] for key in ct_keys]):
        patch.set(color=color, alpha=0.5)
    for whisker in bp['whiskers']:
        whisker.set(color='black', linewidth=1.2, linestyle='-', alpha=0.5)
    for cap in bp['caps']:
        cap.set(linewidth=1.2, alpha=0.6)
    for median in bp['medians']:
        median.set(color='black', linewidth=1.2, alpha=0.5)
    
    # Add jittered data
    for i,p in enumerate(plot_data):
        y = p
        x = np.random.normal(i+1, 0.06, size=len(y))  # Jitter
        ax.plot(x, y, 'k.', alpha=0.1, ms=2)
    
    # Axis cosmetics
    ax.set_xticklabels([archetype_decodedict[key] for key in ct_keys],
                       rotation=45, ha='right', fontsize=16)
    ax.set_ylabel(covariate, fontsize=16)
    
    # Other cosmetics
    ax.set_title("Boxplot of "+covariate+" by archetype")
    
    # Done
    plt.show()
    
    if show_stats:
    
        # Compute p-value with MWU
        print '\n\np-values (Mann-Whitney U-test, two-sided, with Bonferroni):\n'
        for i,a in enumerate(plot_data):
            for j,b in enumerate(plot_data):
                _, pval = stats.mannwhitneyu(a, b, alternative='two-sided')
                pval_bonf = pval * 3
                print archetype_decodedict[ct_keys[i]], 'vs', archetype_decodedict[ct_keys[j]], ':', pval_bonf

        # Compute d-value with Cohen
        print "\n\nEffect size values (Cohen's d, parametric...):\n"
        def cohens_d(pop_a, pop_b):
            m1 = np.mean(pop_a)
            m2 = np.mean(pop_b)
            v1 = np.var(pop_a, ddof=1)
            v2 = np.var(pop_b, ddof=1)
            n1 = len(pop_a)
            n2 = len(pop_b)
            s  = np.sqrt( (v1*(n1-1) + v2*(n2-1) ) / (n1 + n2 - 2) )
            d  = (m2 - m1) / s
            return d
        for i,a in enumerate(plot_data):
            for j,b in enumerate(plot_data):
                d = cohens_d(a, b)
                print archetype_decodedict[ct_keys[i]], 'vs', archetype_decodedict[ct_keys[j]], ':', d

<a id=proba></a>

## 4. Prediction Probabilities

----

In [ ]:
### Tissue consensus map on centroids

# Axis range
xlimit = (-175, 15)
ylimit = (- 20, 20)

# Set interactions
@interact(archetype=[ct for ct in archetype_encodedict.keys() 
                     if ct!='unclassified'])

# Show 
def show_PCA_backmap(archetype='peri'): 

    # Get relevant probability column index
    archetype_idx = archetype_encodedict[archetype] - 1
    
    # Init
    fig, ax = plt.subplots(1, figsize=(12,5))
    
    # Back-mapping plot
    scat = ax.scatter(centroids[:, 2], centroids[:, 1],
                      color=archetype_probas[:, archetype_idx],
                      cmap='viridis', edgecolor='', s=15, alpha=0.75)

    # Cosmetics
    cbar = plt.colorbar(scat, fraction=0.05, pad=0.01)
    cbar.set_label('Probability')
    ax.set_xlim(xlimit)
    ax.set_ylim(ylimit)
    ax.invert_yaxis()  # To match images
    ax.set_xlabel('TFOR x')
    ax.set_ylabel('TFOR y')
    ax.set_title('Centroid backmapping of prediction probability for '+archetype)

    # Done
    plt.show()

In [ ]:
### Mapping onto shape space

# Set interactions
@interact(archetype=[ct for ct in archetype_encodedict.keys() 
                     if ct!='unclassified'],
          use_space=['TFOR','CFOR'],
          PCx=(1, fspace_TFOR_pca.shape[1], 1),
          PCy=(1, fspace_TFOR_pca.shape[1], 1))

# Show 
def show_PCA_backmap(archetype='peri', use_space='TFOR', PCx=1, PCy=2): 
    
    # Select shape space data
    if use_space=='TFOR':
        plot_fspace = fspace_TFOR_pca
    elif use_space=='CFOR':
        plot_fspace = fspace_CFOR_pca
    
    # Select probability column index
    archetype_idx = archetype_encodedict[archetype] - 1
    
    # Prep plot
    plt.figure(figsize=(9,7))

    # Create scatterplot
    scat = plt.scatter(plot_fspace[:, PCx-1], plot_fspace[:, PCy-1],
                       color=archetype_probas[:, archetype_idx],
                       cmap='viridis', edgecolor='', s=15, alpha=0.75)

    # Cosmetics  
    cbar = plt.colorbar(scat, fraction=0.05, pad=0.01)
    cbar.set_label('p('+archetype+')')
    plt.xlabel(use_space+" PC "+str(PCx))
    plt.ylabel(use_space+" PC "+str(PCy))
    plt.title("Archetype '"+archetype+"' prediction probability on shape space")
    plt.show()

In [ ]:
### Correlation of fspace PCs with probabilities

# Set interactions
@interact(archetype=[ct for ct in archetype_encodedict.keys() 
                     if ct!='unclassified'],
          use_space=['TFOR','CFOR'],
          dim=(1, fspace_TFOR_pca.shape[1],1))

# Create boxplot
def corrplot_fspaces(archetype='peri', use_space='TFOR', dim=1):

    # Select shape space data
    if use_space=='TFOR':
        plot_fspace = fspace_TFOR_pca
    elif use_space=='CFOR':
        plot_fspace = fspace_CFOR_pca
    
    # Get relevant probability column index
    archetype_idx = archetype_encodedict[archetype]
    
    # Prep plot
    fig, ax = plt.subplots(1, figsize=(6, 6))
    
    # Create scatterplot of different archetypes
    bp = ax.scatter(archetype_probas[archetype_classes!=archetype_encodedict[archetype], archetype_idx-1], 
                    plot_fspace[archetype_classes!=archetype_encodedict[archetype], dim-1],
                    facecolor='lightgray', 
                    edgecolor='k', linewidth=0.5,
                    alpha=0.3, s=10)
    
    # Create scatterplot of selected archetype
    bp = ax.scatter(archetype_probas[archetype_classes==archetype_encodedict[archetype], archetype_idx-1], 
                    plot_fspace[archetype_classes==archetype_encodedict[archetype], dim-1],
                    facecolor=archetype_colors[archetype_idx], 
                    edgecolor='k', linewidth=0.5,
                    alpha=0.3, s=10)
    
    # Axis cosmetics
    ax.set_xlim([0.0, 1.0])
    ax.set_xlabel("Probability of classification as "+archetype)
    ax.set_ylabel(use_space+" PC "+str(dim))
    
    # Other cosmetics
    ax.set_title("Scatterplot of shape features over archetype probabilities")
    
    # Done
    plt.show()

In [ ]:
### Correlation of probabilities with covariates

# Set interactions
@widgets.interact(archetype=[ct for ct in archetype_encodedict.keys() 
                             if ct!='unclassified'],
                  covariate=covar_names,
                  standardized=['no','z'])

# Plot
def boxplot_covars(archetype='central',
                   covariate='Sphericity',
                   standardized='no'): 
    
    # Select covariate data
    if standardized=='no':
        covar_data = covar_df[covariate]
    elif standardized=='z':
        covar_data = covar_df_z[covariate]

    # Get relevant probability column index
    archetype_idx = archetype_encodedict[archetype]

    # Prep plot
    fig, ax = plt.subplots(1, figsize=(6, 6))
    
    # Create scatterplot of different archetypes
    bp = ax.scatter(archetype_probas[archetype_classes!=archetype_encodedict[archetype], archetype_idx-1], 
                    covar_data[archetype_classes!=archetype_encodedict[archetype]],
                    facecolor='lightgray', 
                    edgecolor='k', linewidth=0.5,
                    alpha=0.3, s=10)
    
    # Create scatterplot of selected archetype
    bp = ax.scatter(archetype_probas[archetype_classes==archetype_encodedict[archetype], archetype_idx-1], 
                    covar_data[archetype_classes==archetype_encodedict[archetype]],
                    facecolor=archetype_colors[archetype_idx], 
                    edgecolor='k', linewidth=0.5,
                    alpha=0.3, s=10)
    
    # Add linear fit
    # TODO... hm... this is probably not worth it...
    
    # Axis cosmetics
    ax.set_xlim([0.0, 1.0])
    ax.set_xlabel("Probability of classification as "+archetype)
    ax.set_ylabel(covariate)
    
    # Other cosmetics
    ax.set_title("Scatterplot of covariates over archetype probability")
    
    # Done
    plt.show()

<a id=archespace></a>

## 5. Archetype Space

----

In [ ]:
### Embed prediction probabilities with PCA

# Embed
archetype_PCA = PCA()
archetype_space = archetype_PCA.fit_transform(archetype_probas)

# Predict
print "  Explained variance ratios:  ", archetype_PCA.explained_variance_ratio_
print "  Created embedding of shape: ", archetype_space.shape

In [ ]:
### 2D Plot of archetype space

# Set interactions
@interact(PCx=(1, archetype_space.shape[1], 1),
          PCy=(1, archetype_space.shape[1], 1))

# Show 
def show_proba_plot(PCx=1, PCy=2): 

    # Prep plot
    plt.figure(figsize=(9,7))

    # Create scatter
    for key in archetype_decodedict.keys():
        mask = archetype_classes==key
        if np.any(mask):
            scat = plt.scatter(archetype_space[mask, PCx-1], archetype_space[mask, PCy-1],
                               color=archetype_colors[key], edgecolor='', 
                               s=10, alpha=0.5, label=archetype_decodedict[key])

    # Cosmetics  
    plt.legend(frameon=False, fontsize=10)
    plt.xlabel("PC "+str(PCx))
    plt.ylabel("PC "+str(PCy))
    plt.title("Archetype space")
    plt.show()

In [ ]:
### 3D plot of archetype space

# Prep plot
fig = plt.figure(figsize=(8,6))
ax  = fig.add_subplot(111, projection='3d')

# Create scatter
for key in archetype_decodedict.keys():
    mask = archetype_classes==key
    if np.any(mask):
        ax.scatter(archetype_space[mask, 0], 
                   archetype_space[mask, 1], 
                   archetype_space[mask, 2],
                   c=archetype_colors[key], linewidth=0, 
                   s=10, alpha=0.2, label=archetype_decodedict[key] )

# Switch off gray panes
ax.w_xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.w_yaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
ax.w_zaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))

# Set view angle
ax.view_init(35, -12)

# Legend (without alpha)  
leg = ax.legend(frameon=False, fontsize=8)
for l in leg.legendHandles:
    l._facecolors[:,-1] = 1.0

# Label size for publication
ax.tick_params(axis='both', which='major', labelsize=14)

# Other cosmetics
ax.set_xlabel("PC 1", labelpad=20, fontsize=16)
ax.set_ylabel("PC 2", labelpad=20, fontsize=16)
ax.set_zlabel("PC 3", labelpad=20, fontsize=16)
plt.axis('equal')
plt.tight_layout()

# Done
plt.show()

In [ ]:
### Mapping of shape space PCs onto 2D plot of archetype space

# Set interactions
@interact(use_space=['TFOR', 'CFOR'],
          dim=(1, fspace_TFOR_pca.shape[1], 1), 
          PCx=(1, archetype_space.shape[1], 1),
          PCy=(1, archetype_space.shape[1], 1),
          vmax_factor=(0.0, 1.0, 0.1),
          show_types=False)

# Show 
def show_proba_fspace_overlay(use_space='CFOR', dim=1, PCx=1, PCy=2, 
                              vmax_factor=1.0, show_types=False): 
    
    # Select shape space data
    if use_space=='TFOR':
        plot_fspace = fspace_TFOR_pca
    elif use_space=='CFOR':
        plot_fspace = fspace_CFOR_pca
    
    # Prep plot
    plt.figure(figsize=(7,6))

    # Create scatter
    if not show_types:
        scat = plt.scatter(archetype_space[:, PCx-1], archetype_space[:, PCy-1],
                           color=plot_fspace[:, dim-1], cmap='viridis',
                           vmax=vmax_factor*np.max(plot_fspace[:, dim-1]),
                           edgecolor='', s=10, alpha=0.7)
        
    # Show the archetypes instead (for reference)
    if show_types:
        for key in archetype_decodedict.keys():
            mask = archetype_classes==key
            if np.any(mask):
                scat = plt.scatter(archetype_space[mask, PCx-1], archetype_space[mask, PCy-1],
                                   color=archetype_colors[key], edgecolor='', 
                                   s=10, alpha=0.5, label=archetype_decodedict[key])
        plt.legend(frameon=False, fontsize=16)

    # Cosmetics  
    plt.xlabel("PC "+str(PCx), fontsize=16)
    plt.ylabel("PC "+str(PCy), fontsize=16)
    plt.title("Overlay of "+use_space+" PC "+str(dim)+" on archetype space")
    plt.gca().tick_params(axis='both', which='major', labelsize=16)
    plt.xticks(np.arange(-0.6, 1.1, 0.3))
    plt.yticks(np.arange(-0.5, 1.1, 0.3))
    plt.tight_layout()

    # Show
    plt.show()

In [ ]:
### Mapping of covariates PCs onto 2D plot of archetype space

# Set interactions
@widgets.interact(covariate=covar_names,
                  standardized=['no','z'],
                  PCx=(1, archetype_space.shape[1], 1),
                  PCy=(1, archetype_space.shape[1], 1),
                  vmax_factor=(0.0, 1.0, 0.1),
                  show_types=False)

# Plot
def show_proba_covar_overlay(covariate='Sphericity', standardized='no',
                             PCx=1, PCy=2, vmax_factor=1.0, show_types=False): 
    
    # Select covariate data
    if standardized=='no':
        covar_data = covar_df[covariate]
    elif standardized=='z':
        covar_data = covar_df_z[covariate]
    
    # Prep plot
    plt.figure(figsize=(9,7))

    # Create scatter
    if not show_types:
        scat = plt.scatter(archetype_space[:, PCx-1], archetype_space[:, PCy-1],
                           color=covar_data, cmap='viridis',
                           vmax=vmax_factor*np.max(covar_data),
                           edgecolor='', s=10, alpha=0.7)
        
    # Show the archetypes instead (for reference)
    if show_types:
        for key in archetype_decodedict.keys():
            mask = archetype_classes==key
            if np.any(mask):
                scat = plt.scatter(archetype_space[mask, PCx-1], archetype_space[mask, PCy-1],
                                   color=archetype_colors[key], edgecolor='', 
                                   s=10, alpha=0.5, label=archetype_decodedict[key])
        plt.legend(frameon=False, fontsize=10)

    # Cosmetics  
    plt.xlabel("PC "+str(PCx))
    plt.ylabel("PC "+str(PCy))
    plt.title("Overlay of "+covariate+" on archetype space")
    plt.gca().tick_params(axis='both', which='major', labelsize=16)
    plt.tight_layout()
    
    # Show
    plt.show()

In [ ]:
### Multi-panel mappings

# Prep
fig, ax = plt.subplots(2, 2, figsize=(12,12), sharex=True, sharey=True)
PCx = 1; PCy=2

# Plot archetypes
for key in archetype_decodedict.keys()[::-1]:
    mask = archetype_classes==key
    if np.any(mask):
        scat = ax[0,0].scatter(archetype_space[mask, PCx-1], archetype_space[mask, PCy-1],
                               color=archetype_colors[key], edgecolor='', s=10, alpha=0.5, 
                               label=archetype_decodedict[key].replace('Cells','').replace('Rosette',''))
        
# Add nice archetype legend
legend = ax[0,0].legend(frameon=False, fontsize=18)
for i in range(len(legend.legendHandles)):
    legend.legendHandles[i]._sizes = [50 for s in legend.legendHandles[i]._sizes]

# Plot CFOR-PC1
dim = 1
vmax_factor=1.0
scat = ax[0,1].scatter(archetype_space[:, PCx-1], archetype_space[:, PCy-1],
                       color=fspace_CFOR_pca[:, dim-1], cmap='viridis',
                       vmax=vmax_factor*np.max(fspace_CFOR_pca[:, dim-1]),
                       edgecolor='', s=10, alpha=0.7)

# Plot TFOR-PC3
dim = 3
vmax_factor=1.0
scat = ax[1,0].scatter(archetype_space[:, PCx-1], archetype_space[:, PCy-1],
                       color=fspace_TFOR_pca[:, dim-1], cmap='viridis',
                       vmax=vmax_factor*np.max(fspace_CFOR_pca[:, dim-1]),
                       edgecolor='', s=10, alpha=0.7)

# Plot Cell Height
covar_data_plot = covar_df['Sphericity']
vmax_factor=1.0
scat = ax[1,1].scatter(archetype_space[:, PCx-1], archetype_space[:, PCy-1],
                       color=covar_data_plot, cmap='viridis',
                       vmax=vmax_factor*np.max(covar_data_plot),
                       edgecolor='', s=10, alpha=0.7)
  
# Cosmetics
for axx in ax.flatten():
    axx.set_xticks(np.arange(-0.5, 1.1, 0.25))
    axx.set_yticks(np.arange(-0.5, 1.1, 0.25))
    axx.set_xlim([-0.7, 0.9]); axx.set_ylim([-0.5, 1.0])
    axx.tick_params(axis='both', which='major', labelsize=18)
plt.tight_layout()

# Show
plt.show()

In [ ]:
### Multi-panel mappings (alternative)

# Prep
fig, ax = plt.subplots(1,3, figsize=(12,4), sharex=True, sharey=True)
PCx = 1; PCy=2

# Plot TFOR-PC1
dim = 1
vmax_factor=1.0
scat = ax[0].scatter(archetype_space[:, PCx-1], archetype_space[:, PCy-1],
                     color=fspace_TFOR_pca[:, dim-1], cmap='viridis',
                     vmax=vmax_factor*np.max(fspace_TFOR_pca[:, dim-1]),
                     edgecolor='', s=7, alpha=0.7)

# Plot TFOR-PC3
dim = 3
vmax_factor=1.0
scat = ax[1].scatter(archetype_space[:, PCx-1], archetype_space[:, PCy-1],
                     color=fspace_TFOR_pca[:, dim-1], cmap='viridis',
                     vmax=vmax_factor*np.max(fspace_TFOR_pca[:, dim-1]),
                     edgecolor='', s=7, alpha=0.7)

# Plot CFOR-PC1
dim = 1
vmax_factor=1.0
scat = ax[2].scatter(archetype_space[:, PCx-1], archetype_space[:, PCy-1],
                     color=fspace_CFOR_pca[:, dim-1], cmap='viridis',
                     vmax=vmax_factor*np.max(fspace_CFOR_pca[:, dim-1]),
                     edgecolor='', s=7, alpha=0.7)

# Cosmetics
for axx in ax:
    axx.set_xticks(np.arange(-0.5, 1.1, 0.5))
    axx.set_yticks(np.arange(-0.5, 1.1, 0.5))
    axx.set_xlim([-0.7, 0.9]); axx.set_ylim([-0.5, 1.0])
    axx.tick_params(axis='both', which='major', labelsize=18)
    axx.set_xticklabels(['', '', ''])
plt.tight_layout()

# Show
plt.show()

----
[back to top](#top)